# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,74.46,62,94,26.19,PN,1711845573
1,1,tiksi,71.6872,128.8694,9.03,77,100,15.82,RU,1711845614
2,2,huarmey,-10.0681,-78.1522,74.14,83,90,7.56,PE,1711845636
3,3,qaqortoq,60.7167,-46.0333,23.92,86,88,1.95,GL,1711845583
4,4,vilyuysk,63.7553,121.6247,5.00,91,61,4.00,RU,1711845807


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
# YOUR CODE HERE

humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["City", "Humidity"],
    alpha = .5
)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df[(city_data_df["Humidity"] < 75) \
                    & (city_data_df["Max Temp"] > 60) \
                    & (city_data_df["Max Temp"] < 75) \
                    & (city_data_df["Cloudiness"] < 50)]



# Drop any rows with null values
ideal_city_data_clean = ideal_city_data_df.dropna()



# Display sample data
ideal_city_data_clean.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,34,hendek,40.7994,30.7481,61.66,53,6,5.93,TR,1711845813
42,42,fort bragg,35.1390,-79.0060,73.24,26,0,10.36,US,1711845816
65,65,tamanrasset,22.7850,5.5228,62.51,10,0,5.75,DZ,1711845821
106,106,sedalia,38.7045,-93.2283,69.71,40,40,4.61,US,1711845631
107,107,danville,36.5860,-79.3950,70.18,24,0,6.91,US,1711845668


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_clean.copy()
hotel_df = hotel_df.drop(["City_ID", "Max Temp", "Cloudiness", "Wind Speed", "Date"], axis = 1)


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
34,hendek,40.7994,30.7481,53,TR,
42,fort bragg,35.1390,-79.0060,26,US,
65,tamanrasset,22.7850,5.5228,10,DZ,
106,sedalia,38.7045,-93.2283,40,US,
107,danville,36.5860,-79.3950,24,US,
119,hermanus,-34.4187,19.2345,58,ZA,
163,whakatane,-37.9585,176.9854,62,NZ,
192,port lincoln,-34.7333,135.8667,53,AU,
201,demre,36.2537,29.9777,54,TR,
203,bredasdorp,-34.5322,20.0403,66,ZA,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":1
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    #print(params)
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

     # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    #Convert the API response to JSON format
    name_address = name_address.json()  

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hendek - nearest hotel: Öğretmenevi
fort bragg - nearest hotel: Airborne Inn Lodging
tamanrasset - nearest hotel: فندق الأمان
sedalia - nearest hotel: Hotel Bothwell
danville - nearest hotel: Holiday Inn Express Danville
hermanus - nearest hotel: Aloe guest house
whakatane - nearest hotel: Whakatane Hotel
port lincoln - nearest hotel: Boston Hotel
demre - nearest hotel: Hotel Kekova
bredasdorp - nearest hotel: Victoria Hotel
lanzhou - nearest hotel: 西园宾馆
greenville - nearest hotel: Quality Inn
gulf shores - nearest hotel: Beachside Resort
codrington - nearest hotel: No hotel found
mount gambier - nearest hotel: The Old Mount Gambier Gaol
prien - nearest hotel: Golden Nugget Lake Charles Hotel & Casino
phalodi - nearest hotel: LAL NIWAS
fort myers shores - nearest hotel: Comfort Inn Fort Myers
durango - nearest hotel: No hotel found
edgewater - nearest hotel: Best Western Edgewater Inn
ghat - nearest hotel: فندق تاسيلي
guimar - nearest hotel: No hotel found
hamilto

,City,Lat,Lng,Humidity,Country,Hotel Name
34,hendek,40.7994,30.7481,53,TR,Öğretmenevi
42,fort bragg,35.1390,-79.0060,26,US,Airborne Inn Lodging
65,tamanrasset,22.7850,5.5228,10,DZ,فندق الأمان
106,sedalia,38.7045,-93.2283,40,US,Hotel Bothwell
107,danville,36.5860,-79.3950,24,US,Holiday Inn Express Danville
119,hermanus,-34.4187,19.2345,58,ZA,Aloe guest house
163,whakatane,-37.9585,176.9854,62,NZ,Whakatane Hotel
192,port lincoln,-34.7333,135.8667,53,AU,Boston Hotel
201,demre,36.2537,29.9777,54,TR,Hotel Kekova
203,bredasdorp,-34.5322,20.0403,66,ZA,Victoria Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"],
    alpha = .5
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)